# Creating a custom Index in Pandas

The pandas.Datetimeindex is the jam because of the extra properties that access portions of the time.  I'd really like a `pathlib` version of this, `...` and a `luigi` target version.

In [1]:
from pathlib import Path, os
from pandas import Index, Series, DataFrame; import pandas as pd
from toolz.curried.operator import *; from toolz.curried import *

## Using typing information as data

`pathlib` has an annotated type file in `typeshed`.  The script below makes the annotations importable.

In [2]:
%%file pathlib_types.py
from pathlib import Path, PurePath
exec(next(
    Path(Path.cwd().parent.parent/'typeshed/stdlib/').rglob('pathlib*')).read_text(), globals())

Overwriting pathlib_types.py


In [3]:
import pathlib_types as types
target = types.Path

The  __annotations__ for `Path`'s attributes are held in `target`; _this is a function parameter later_.

In [4]:
def method(x, target=target):
    """Get annotations from bound methods and properties.s"""
    if isinstance(x, property):
        return getattr(target, x.fget.__name__).fget.__annotations__['return']
    attr = getattr(x, '__name__', None)
    if attr in dir(target):
        return getattr(getattr(target, attr), '__annotations__', {}).get('return', None)

`returns` is an object of the attributes and their return `type`.

In [5]:
returns = pipe(
    target, dir, filter(compose(str.isalpha, first)), 
    map(juxt(identity, curry(getattr)(target))),
    dict, valmap(method))

Series(returns).astype(str).to_frame().reset_index().set_index(0).sort_index().sample(10).T

,<class 'str'>,None,<class 'int'>,None,<class 'bool'>,<class 'os.stat_result'>,None,<class 'bool'>,~_P,None
index,read_text,rmdir,write_bytes,name,is_socket,stat,lchmod,is_reserved,joinpath,replace


> A small sample of the type to attribute mapping.

In [6]:
from functools import wraps

class PathIndex(Index):
    def __new__(cls, data=None, dtype=None, copy=False, name=None, fastpath=False, tupleize_cols=True, **kwargs):
        if isiterable(data) and not isinstance(data, Index):
            data = cls._coerce_to_ndarray(map(Path, data))
        result = cls._simple_new(data, name, dtype, copy=copy, fastpath=fastpath, tupleize_cols=tupleize_cols, **kwargs)
        return result
    
    def __getattribute__(self, name):
        try:
            return super().__getattribute__(name)
        except AttributeError as e:
            attr = getattr(Path, name, None)
            
            # The name is wrong!
            if not attr: raise e
            
            cls = type_mapper[method(attr)]
            
            # There is no way to interpret the attribute
            if cls is False: raise NotImplemented
            
            
            if isinstance(attr, property): #  return computed values, not a callable.
                return (self if cls is None else cls(self.map(attrgetter(name))))

            # wrap a callable for later.
            @wraps(attr)
            def mapper(*args, **kwargs):                        
                if cls is None: return self        
                result = cls(self.map(partial(lambda v: attr(v, *args, **kwargs))))
                if isinstance(result, (Series, DataFrame)):
                    result.index = self
                return result
            return mapper
            
    def __truediv__(self, value):
        return type(self)(self.map(flip(Path.__truediv__)(value)))
    
    def __rtruediv__(self, value):
        return type(self)(self.map(flip(Path.__rtruediv__)(value)))

    def __dir__(self):
        """Update the available attribute."""
        return list(super().__dir__()) + dir(Path)

In [7]:
type_mapper = {
    bool: Index,
    types._P: PathIndex,
    None: None, 
    types.IO[types.Any]: False,
    types.Generator[Path, None, None]: compose(PathIndex, list, concat),
    types.Sequence[types._P]: compose(PathIndex, list, concat),
    int: Index, os.stat_result: Index,
    str: Series, bytes: Series,
}

## A test drive of `PathIndex`


In [8]:
assert all(map(contains(type_mapper.keys()), returns.values()))

if __name__ == '__main__':
    i = PathIndex(['nbd', 'fidget', '.'])
    print(i.exists(), ('../..'/i).exists())

Index([False, False, True], dtype='object') Index([True, True, True], dtype='object')


In [9]:
if __name__ == '__main__':
    df = ('../..'/i)[:2].glob('*.ipynb').read_text().apply(
        __import__('nbformat').reads, args=[4]
    ).apply(
        compose(DataFrame, get('cells'))
    ).pipe(compose(pd.concat, dict, Series.iteritems))
    print(i.exists(), ('../..'/i).exists())
__name__ == '__main__' and df

Index([False, False, True], dtype='object') Index([True, True, True], dtype='object')


cell_type  execution_count             metadata  \
../../fidget/readme.ipynb  0   markdown              NaN                   {}   
                           1       code             17.0                   {}   
                           2   markdown              NaN                   {}   
                           3       code             47.0                   {}   
                           4       code             53.0                   {}   
                           5   markdown              NaN                   {}   
                           6   markdown              NaN                   {}   
                           7       code             64.0                   {}   
                           8       code             65.0                   {}   
                           9       code              1.0                   {}   
                           10      code              NaN  {'collapsed': True}   
../../fidget/testing.ipynb 0       code             30.0  {'collapsed': True}   
                           1       code             31.0                   {}   
                           2       code              3.0                   {}   
                           3       code              4.0                   {}   
                           4       code              5.0                   {}   
                           5       code              6.0                   {}   
                           6       code              7.0                   {}   
                           7       code              8.0                   {}   
                           8       code              9.0                   {}   
                           9       code             10.0                   {}   
                           10      code             11.0                   {}   
                           11      code             12.0  {'collapsed': True}   
                           12      code             13.0                   {}   
                           13      code             14.0                   {}   
                           14      code             15.0  {'collapsed': True}   
                           15      code             16.0                   {}   
                           16      code             17.0                   {}   
                           17  markdown              NaN                   {}   
                           18      code             18.0                   {}   
...                                 ...              ...                  ...   
../../nbd/nbd.ipynb        9       code            112.0                   {}   
                           10  markdown              NaN                   {}   
                           11  markdown              NaN                   {}   
                           12      code              6.0                   {}   
                           13  markdown              NaN                   {}   
                           14      code              7.0                   {}   
                           15      code              8.0                   {}   
                           16  markdown              NaN                   {}   
                           17      code              9.0                   {}   
                           18  markdown              NaN                   {}   
                           19      code             10.0                   {}   
                           20  markdown              NaN                   {}   
                           21      code            111.0                   {}   
                           22  markdown              NaN                   {}   
                           23      code              NaN                   {}   
../../nbd/readme.ipynb     0   markdown              NaN                   {}   
                           1   markdown              NaN                   {}   
                           2   markdown              NaN     

### Notes

* Logic works.